In [1]:
# First I am setting up the notebooke with the necessary imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings("ignore")

from acquire import get_telco_data

In [2]:
# now I am naming my dataframe telco and calling it from the csv file made in the acquire file.
telco= pd.read_csv("telco.csv")
telco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                7043 non-null   int64  
 1   payment_type_id           7043 non-null   int64  
 2   internet_service_type_id  7043 non-null   int64  
 3   contract_type_id          7043 non-null   int64  
 4   customer_id               7043 non-null   object 
 5   gender                    7043 non-null   object 
 6   senior_citizen            7043 non-null   int64  
 7   partner                   7043 non-null   object 
 8   dependents                7043 non-null   object 
 9   tenure                    7043 non-null   int64  
 10  phone_service             7043 non-null   object 
 11  multiple_lines            7043 non-null   object 
 12  online_security           7043 non-null   object 
 13  online_backup             7043 non-null   object 
 14  device_p

In [3]:
#ensure that any duplicate customers are removed from the dataframe
telco.drop_duplicates(subset=['customer_id'], keep='last')

,Unnamed: 0,payment_type_id,internet_service_type_id,contract_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type
0,0,2,1,1,0003-MKNFE,Male,0,No,No,9,...,No,No,Yes,No,59.90,542.4,No,Month-to-month,DSL,Mailed check
1,1,4,1,1,0013-MHZWF,Female,0,No,Yes,9,...,Yes,Yes,Yes,Yes,69.40,571.45,No,Month-to-month,DSL,Credit card (automatic)
2,2,1,1,1,0015-UOCOJ,Female,1,No,No,7,...,No,No,No,Yes,48.20,340.35,No,Month-to-month,DSL,Electronic check
3,3,1,1,1,0023-HGHWL,Male,1,No,No,1,...,No,No,No,Yes,25.10,25.1,Yes,Month-to-month,DSL,Electronic check
4,4,3,1,1,0032-PGELS,Female,0,Yes,Yes,1,...,No,No,No,No,30.50,30.5,Yes,Month-to-month,DSL,Bank transfer (automatic)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,7038,4,3,3,9950-MTGYX,Male,0,Yes,Yes,28,...,No internet service,No internet service,No internet service,Yes,20.30,487.95,No,Two year,None,Credit card (automatic)
7039,7039,2,3,3,9953-ZMKSM,Male,0,No,No,63,...,No internet service,No internet service,No internet service,No,25.25,1559.3,No,Two year,None,Mailed check
7040,7040,4,3,3,9964-WBQDJ,Female,0,Yes,No,71,...,No internet service,No internet service,No internet service,Yes,24.40,1725.4,No,Two year,None,Credit card (automatic)
7041,7041,3,3,3,9972-EWRJS,Female,0,Yes,Yes,67,...,No internet service,No internet service,No internet service,Yes,19.25,1372.9,No,Two year,None,Bank transfer (automatic)


In [4]:
#making single variable columns
telco['years_tenure'] = telco.tenure / 12
telco['is_family']=telco["partner" or "dependents"] == 'Yes'
telco['is_senior']=telco["senior_citizen"]== "Yes"
telco['has_phones']= telco['phone_service' or 'multiple_lines']== 'Yes'
telco['has_paperless_billing']= telco['paperless_billing']=='Yes'
telco['has_streaming']= telco["streaming_tv" or "streaming_movies"] == 'Yes'
telco['has_support_features']= telco['device_protection' or 'tech_support']=='Yes'
telco['has_security_features']= telco['online_security' or 'online_backup'] =='Yes'

In [5]:
# removing id columns and string variable columns
telco= telco.drop(columns=["paperless_billing","device_protection", "tech_support", "senior_citizen","phone_service", "streaming_tv", "streaming_movies", "partner","dependents","online_security", "online_backup","Unnamed: 0","customer_id","payment_type_id","tenure","contract_type_id", "internet_service_type_id"])
telco.head()

,gender,multiple_lines,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type,years_tenure,is_family,is_senior,has_phones,has_paperless_billing,has_streaming,has_support_features,has_security_features
0,Male,Yes,59.9,542.4,No,Month-to-month,DSL,Mailed check,0.750000,False,False,True,False,False,False,False
1,Female,No,69.4,571.45,No,Month-to-month,DSL,Credit card (automatic),0.750000,False,False,True,True,True,False,False
2,Female,No,48.2,340.35,No,Month-to-month,DSL,Electronic check,0.583333,False,False,True,True,False,False,True
3,Male,No phone service,25.1,25.1,Yes,Month-to-month,DSL,Electronic check,0.083333,False,False,False,True,False,False,False
4,Female,No phone service,30.5,30.5,Yes,Month-to-month,DSL,Bank transfer (automatic),0.083333,True,False,False,False,False,False,True


In [6]:
#making dummy variables
telco_dummies = pd.get_dummies(telco.churn, drop_first=True)
telco_dummies.head(3)

,Yes
0,0
1,0
2,0


In [7]:
# attaching the dummy variables onto the data frame
telco = pd.concat([telco, telco_dummies], axis=1)
#renaming the column from yes to is_churn for clarity
telco= telco.rename(columns={'Yes': 'is_churn'})
#ensuring that the dummy variables are attached to the dataframe
telco.head(3)

,gender,multiple_lines,monthly_charges,total_charges,churn,contract_type,internet_service_type,payment_type,years_tenure,is_family,is_senior,has_phones,has_paperless_billing,has_streaming,has_support_features,has_security_features,is_churn
0,Male,Yes,59.9,542.4,No,Month-to-month,DSL,Mailed check,0.750000,False,False,True,False,False,False,False,0
1,Female,No,69.4,571.45,No,Month-to-month,DSL,Credit card (automatic),0.750000,False,False,True,True,True,False,False,0
2,Female,No,48.2,340.35,No,Month-to-month,DSL,Electronic check,0.583333,False,False,True,True,False,False,True,0


In [8]:
#checking the Dtypes and Columns of my clean dataframe
telco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   gender                 7043 non-null   object 
 1   multiple_lines         7043 non-null   object 
 2   monthly_charges        7043 non-null   float64
 3   total_charges          7043 non-null   object 
 4   churn                  7043 non-null   object 
 5   contract_type          7043 non-null   object 
 6   internet_service_type  7043 non-null   object 
 7   payment_type           7043 non-null   object 
 8   years_tenure           7043 non-null   float64
 9   is_family              7043 non-null   bool   
 10  is_senior              7043 non-null   bool   
 11  has_phones             7043 non-null   bool   
 12  has_paperless_billing  7043 non-null   bool   
 13  has_streaming          7043 non-null   bool   
 14  has_support_features   7043 non-null   bool   
 15  has_